## Import Dependencies

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
from urllib.parse import quote_plus
import os
from dotenv import load_dotenv



In [2]:

load_dotenv()

DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD', '')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

# Safely encode special characters in the password for the connection string
encoded_password = quote_plus(DB_PASSWORD)

SQLALCHEMY_DATABASE_URL = f"mysql+pymysql://{DB_USER}:{encoded_password}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(SQLALCHEMY_DATABASE_URL)

In [3]:
chapters_df = pd.read_sql("""
    SELECT 
        id AS chapter_id,
        chapter_title AS chapter_name,
        course_id,
        owner_id AS user_id
    FROM chapters
""", engine)

chapters_df


,chapter_id,chapter_name,course_id,user_id
0,1,Algebraic Expression,1,2
1,2,Linear Expression,1,2
2,3,Gravitational Force,2,2
3,5,How to talk to strangers,6,4
4,6,Introduction to programming,4,2
5,7,Exponential expression,1,2


In [4]:
time_df = pd.read_sql("""
    SELECT
        chapter_id,
        owner_id AS user_id,
        SUM(CASE WHEN activity_type = 'view_content' THEN duration_seconds ELSE 0 END) AS view_content,
        SUM(CASE WHEN activity_type = 'summary' THEN duration_seconds ELSE 0 END) AS time_summary,
        SUM(CASE WHEN activity_type = 'ask_question' THEN duration_seconds ELSE 0 END) AS time_ask,
        SUM(CASE WHEN activity_type = 'mcq' THEN duration_seconds ELSE 0 END) AS time_mcq,
        MAX(session_end) AS last_activity
    FROM learning_sessions
    WHERE is_valid = 1
    GROUP BY chapter_id, owner_id
""", engine)

time_df


,chapter_id,user_id,view_content,time_summary,time_ask,time_mcq,last_activity
0,1,2,38.0,26.0,0.0,30.0,2025-12-31 02:37:08
1,2,2,0.0,5.0,5.0,11.0,2025-12-30 20:58:25
2,3,2,0.0,0.0,0.0,24.0,2025-12-30 21:04:04
3,6,2,0.0,0.0,0.0,11.0,2025-12-30 22:40:20


In [5]:
mcq_df = pd.read_sql("""
    SELECT
        chapter_id,
        owner_id AS user_id,
        COUNT(*) AS mcq_attempts,
        AVG(score_percentage) AS mcq_avg_score,
        MAX(attempted_at) AS last_mcq_attempt
    FROM mcq_attempts
    GROUP BY chapter_id, owner_id
""", engine)

mcq_df


,chapter_id,user_id,mcq_attempts,mcq_avg_score,last_mcq_attempt
0,1,2,2,50.0,2025-12-30 14:57:49
1,2,2,1,80.0,2025-12-30 14:59:48
2,3,2,2,70.0,2025-12-30 15:04:52


In [6]:
df = chapters_df \
    .merge(time_df, on=["chapter_id", "user_id"], how="left") \
    .merge(mcq_df, on=["chapter_id", "user_id"], how="left")


In [7]:
df.fillna({
    "view_content":0,
    "time_summary": 0,
    "time_ask": 0,
    "time_mcq": 0,
    "mcq_attempts": 0,
    "mcq_avg_score": 0
}, inplace=True)


In [8]:
df["total_time"] = df["time_summary"] + df["time_ask"] + df["time_mcq"] + df["view_content"]

df["score_efficiency"] = df["mcq_avg_score"] / (df["total_time"] + 1)

# Use timezone-naive timestamp to match database timestamps
now = pd.Timestamp.now(tz=None)
df["inactive_days"] = (now - pd.to_datetime(df["last_activity"])).dt.days
df["inactive_days"].fillna(999, inplace=True)


/var/folders/j2/bny1m67s2_d0jg2q7zt0v2nh0000gn/T/ipykernel_52052/612110318.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["inactive_days"].fillna(999, inplace=True)


In [9]:
final_df = df[[
    "user_id",
    "course_id",
    "chapter_id",
    "chapter_name",
    "view_content",
    "time_summary",
    "time_ask",
    "time_mcq",
    "total_time",
    "mcq_attempts",
    "mcq_avg_score",
    "score_efficiency",
    "inactive_days"
]]

final_df


,user_id,course_id,chapter_id,chapter_name,view_content,time_summary,time_ask,time_mcq,total_time,mcq_attempts,mcq_avg_score,score_efficiency,inactive_days
0,2,1,1,Algebraic Expression,38.0,26.0,0.0,30.0,94.0,2.0,50.0,0.526316,-1.0
1,2,1,2,Linear Expression,0.0,5.0,5.0,11.0,21.0,1.0,80.0,3.636364,-1.0
2,2,2,3,Gravitational Force,0.0,0.0,0.0,24.0,24.0,2.0,70.0,2.800000,-1.0
3,4,6,5,How to talk to strangers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,999.0
4,2,4,6,Introduction to programming,0.0,0.0,0.0,11.0,11.0,0.0,0.0,0.000000,-1.0
5,2,1,7,Exponential expression,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,999.0


In [10]:
final_df["needs_attention"] = (
    (final_df["mcq_avg_score"] < 60) |
    (final_df["inactive_days"] > 7)
).astype(int)


/var/folders/j2/bny1m67s2_d0jg2q7zt0v2nh0000gn/T/ipykernel_52052/683948404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["needs_attention"] = (


In [11]:
final_df['score_efficiency'] = final_df.pop('score_efficiency')

/var/folders/j2/bny1m67s2_d0jg2q7zt0v2nh0000gn/T/ipykernel_52052/1136161457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['score_efficiency'] = final_df.pop('score_efficiency')


In [12]:
final_df

,user_id,course_id,chapter_id,chapter_name,view_content,time_summary,time_ask,time_mcq,total_time,mcq_attempts,mcq_avg_score,inactive_days,needs_attention,score_efficiency
0,2,1,1,Algebraic Expression,38.0,26.0,0.0,30.0,94.0,2.0,50.0,-1.0,1,0.526316
1,2,1,2,Linear Expression,0.0,5.0,5.0,11.0,21.0,1.0,80.0,-1.0,0,3.636364
2,2,2,3,Gravitational Force,0.0,0.0,0.0,24.0,24.0,2.0,70.0,-1.0,0,2.800000
3,4,6,5,How to talk to strangers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999.0,1,0.000000
4,2,4,6,Introduction to programming,0.0,0.0,0.0,11.0,11.0,0.0,0.0,-1.0,1,0.000000
5,2,1,7,Exponential expression,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999.0,1,0.000000


In [13]:
X = final_df.drop(columns=["user_id", "course_id", "chapter_id", "chapter_name", "score_efficiency" ], axis=1)
y = final_df['score_efficiency']

In [14]:
X

,view_content,time_summary,time_ask,time_mcq,total_time,mcq_attempts,mcq_avg_score,inactive_days,needs_attention
0,38.0,26.0,0.0,30.0,94.0,2.0,50.0,-1.0,1
1,0.0,5.0,5.0,11.0,21.0,1.0,80.0,-1.0,0
2,0.0,0.0,0.0,24.0,24.0,2.0,70.0,-1.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999.0,1
4,0.0,0.0,0.0,11.0,11.0,0.0,0.0,-1.0,1
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999.0,1


In [15]:
y

0    0.526316
1    3.636364
2    2.800000
3    0.000000
4    0.000000
5    0.000000
Name: score_efficiency, dtype: float64